In [1]:
import pandas as pd
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm



/home/supriya/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# # Load job descriptions from "job_descriptions.csv"
# job_descriptions_df = pd.read_csv("job_descriptions.csv")
# job_descriptions = job_descriptions_df["Job Description"].tolist()


In [3]:

# # Load merged data from "merged.csv"
# merged_df = pd.read_csv("merged.csv")
# if "Top Skills" in merged_df.columns:
#     resume_skills = merged_df["Top Skills"].fillna('').str.split(',').apply(lambda x: [s.strip() for s in x]).tolist()
# elif "Skills" in merged_df.columns:
#     resume_skills = merged_df["Skills"].fillna('').str.split(',').apply(lambda x: [s.strip() for s in x]).tolist()
# else:
#     raise ValueError("No 'Top Skills' or 'Skills' column found in merged.csv")


In [4]:

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")


In [5]:

# Function to preprocess and get embeddings for text
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling
    return embeddings.numpy()  # Convert to numpy array


In [6]:

# # Create a DataFrame to store results
# results_df = pd.DataFrame(columns=["Job Description", "Selected Resumes", "Similarity Scores"])


In [7]:

# # Iterate through job descriptions
# for job_description in tqdm(job_descriptions, desc="Processing Job Descriptions"):
#     job_description_embeddings = get_embeddings(job_description)

#     top_resumes = []
#     top_similarity_scores = []

#     # Calculate cosine similarity scores for each resume individually
#     for resume_skill in resume_skills:
#         resume_embeddings = get_embeddings(' '.join(resume_skill))
#         similarity_score = calculate_cosine_similarity(job_description_embeddings, resume_embeddings)
#         top_resumes.append(resume_skill)  # Append the skill as a representation of the resume
#         top_similarity_scores.append(similarity_score[0][0])

#     # Find the top 3 resumes with highest similarity scores
#     top_indices = np.argsort(top_similarity_scores)[-3:][::-1]
#     top_resumes = [top_resumes[i] for i in top_indices]
#     top_similarity_scores = [top_similarity_scores[i] for i in top_indices]

#     new_row = {
#         "Job Description": job_description,
#         "Selected Resumes": ', '.join([' '.join(resume) for resume in top_resumes]),
#         "Similarity Scores": ', '.join(map(str, top_similarity_scores))
#     }
#     results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)


In [8]:
import pandas as pd

In [9]:
d = {'company name': ['Google'], 'Job Description': ['We are seeking a skilled Data Scientist to join our team. As a Data Scientist, you will be responsible for interpreting complex data sets to provide actionable insights and solutions. Your role will involve developing algorithms, conducting statistical analysis, and creating predictive models to optimize decision-making processes.'], 'Position Title': ['Data Scientist']}
df = pd.DataFrame(data=d)

In [10]:
df

,company name,Job Description,Position Title
0,Google,We are seeking a skilled Data Scientist to joi...,Data Scientist


In [11]:
jd = df['Job Description'].iloc[0]

In [12]:
print(jd)

We are seeking a skilled Data Scientist to join our team. As a Data Scientist, you will be responsible for interpreting complex data sets to provide actionable insights and solutions. Your role will involve developing algorithms, conducting statistical analysis, and creating predictive models to optimize decision-making processes.


In [13]:
import os
import csv
from PyPDF2 import PdfReader
import spacy
import re
import yake

In [14]:
def extract_text_from_pdf(pdf_file_path):
    try:
        reader = PdfReader(pdf_file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        print("Error:", str(e))
        return None


In [15]:
text = extract_text_from_pdf('Supriya_Resume.pdf')

In [16]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\n', ' ', text)

    return text 


In [17]:
p_text = preprocess_text(text)

In [18]:
# from keybert import KeyBERT
# kw_model = KeyBERT()

In [19]:
def extract_skills_keywords(text):
    keyword_extractor = yake.KeywordExtractor()

    skill_keywords_extracted = keyword_extractor.extract_keywords(text)

    skill_keywords = [keyword for keyword, score in skill_keywords_extracted]

    return skill_keywords

In [20]:
from pyresparser import ResumeParser
import warnings


In [21]:

warnings.filterwarnings("ignore", category=UserWarning)

In [22]:

data = ResumeParser("Supriya_Resume.pdf").get_extracted_data()

OSError: [E053] Could not read config file from /home/supriya/anaconda3/lib/python3.11/site-packages/en_core_web_sm/en_core_web_sm-2.3.1/config.cfg

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')
 
SKILLS_DB = [
    'machine learning',
    'data science',
    'python',
    'word',
    'excel',
    'english',
    'react js', 
    'node.js', 
    'nest js',
    'express', 
    'nest.js',  
    'git', 
    'postgresql',
    'firebase', 
    'redis', 
    'mongodb', 
    'docker', 
    'cloud services', 
    'chakra ui',
]


 
 
def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)
 
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
 
    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
 
    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
 
    # we create a set to keep the results in.
    found_skills = set()
 
    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)
 
    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)
 
    return found_skills
 
 


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/supriya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
text_n = extract_text_from_pdf('nimesh_CV.pdf')

In [ ]:
p_text_n = preprocess_text(text_n) 

In [ ]:
print(p_text_n)

skills summary github projects weather app  github  https   github com x  nimesh weather sorting visualizer  github  https   github com x  nimesh sorting visualizerrobinhood ai  github  https   github com x  nimesh robinhood_ai canteen management system  github  https   github com x  nimesh canteen management  system and more in my github nayabazar  kathmandu9869166973 https   github com x nimesh linkedin com in x nimesh   nimesh11 com npx nimesh7 gmail comcontact me at translate creative ui design to good quality code  integrated restful apis and graphql endpoints react js  diagonal technology  sankhamul   may 2022   sep 2022founder  wordpress administrator the rise online   jan 2020   2021work experience infodevelopers   sanepa   nov 2022   jun 2023node js developed the backend infrastructure for a cutting edge fintech application  nccs  national college of computer science   paknajol  kathmandu   neb  2  ashad 2074   2076prime college  khusibun  kathmandu   bca  bachelor in computer

{'chakra ui',
 'cloud services',
 'docker',
 'express',
 'firebase',
 'git',
 'mongodb',
 'nest js',
 'postgresql',
 'react js',
 'redis'}

In [ ]:
extract_skills(p_text)

{'data science', 'machine learning', 'python'}

In [ ]:
# Load spaCy language model
nlp = spacy.load("en_core_web_sm")

# Set spaCy language model as the default for Pyresparser
# ResumeParser.DEFAULT_MODEL_PATH = nlp

# Parse resume using Pyresparser
data = ResumeParser('/home/supriya/Desktop/R-JD-matching/Supriya_Resume.pdf').get_extracted_data()

/home/supriya/anaconda3/lib/python3.11/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'en_training' (0.0.0) specifies an under-constrained spaCy version requirement: >=2.1.4. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /home/supriya/anaconda3/lib/python3.11/site-packages/pyresparser/config.cfg

In [ ]:
print(data)

In [ ]:
from affinda import AffindaAPI, TokenCredential

# Define your Affinda API token credential
credential = TokenCredential(token="aff_00307970892efee73b267c62144d310c41848670")

# Create an Affinda API client
client = AffindaAPI(credential=credential)

# Open the resume file in binary mode
with open("/home/supriya/Desktop/R-JD-matching/Supriya_Resume.pdf", "rb") as f:
    try:
        # Create a resume extraction task using the Affinda API
        response = client.create_resume()
        print("Resume extraction task created successfully!")
        print("Task ID:", response['task_id'])
    except Exception as e:
        print("An error occurred:", e)


An error occurred: 'AffindaAPI' object has no attribute 'create_resume'


In [ ]:
from resume_parser import resumeparse

data = resumeparse.read_file('/home/supriya/Desktop/R-JD-matching/Supriya_Resume.pdf')

/home/supriya/anaconda3/lib/python3.11/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'en_training' (0.0.0) specifies an under-constrained spaCy version requirement: >=2.2.4. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /home/supriya/anaconda3/lib/python3.11/site-packages/resume_parser/degree/model/config.cfg

In [ ]:
# from keybert import KeyBERT
# kw_model = KeyBERT()
# keywords = kw_model.extract_keywords(doc)

In [ ]:
extract_skills_keywords(p_text)

['thapa', 'supriya', 'expertise', 'ai', 'projects']

In [ ]:
def extract_skills_from_resume(resume_text):
    preprocessed_text = preprocess_text(resume_text)

    extracted_skills = extract_skills_keywords(preprocessed_text)

    return extracted_skills

In [ ]:
skills = extract_skills_from_resume(text)

In [ ]:
print(skills)

['thapa', 'supriya', 'expertise', 'ai', 'projects']


In [ ]:

# # Save results to a CSV file
# results_df.to_csv("job_description_matching_results.csv", index=False)


In [ ]:
job_description_embeddings = get_embeddings(jd)


In [ ]:
resume_embeddings = get_embeddings(skills)


In [ ]:

# Function to calculate cosine similarity between two sets of embeddings
def calculate_cosine_similarity(embeddings1, embeddings2):
    return cosine_similarity(np.array(embeddings1), np.array(embeddings2))


In [ ]:
similarity_score = calculate_cosine_similarity(job_description_embeddings, resume_embeddings)


In [ ]:
print(similarity_score)

[[0.39816195 0.460676   0.4731068  0.41287562 0.44128025]]


In [ ]:
print(similarity_score[0][0])

0.39816195


In [ ]:
def load_model():
  nlp = spacy.load('generated_model/model-best')
  return nlp

In [ ]:

def function_1(file_path):
  '''This function takes the pdf file number from the user and then choose the corresponding resumes 
  from the resume file and then give the result'''
  text = extract_text_from_pdf(file_path)
  # Preprocessing the text
  text = preprocess_text(text)
  # Load the model
  nlp = load_model()
  for doc in nlp.pipe([text], disable=["tagger", "parser"]):
    for ent in doc.ents:
        text_name = re.sub('[^A-Za-z0-9]+', ' ', ent.text).strip()
        print((text_name,ent.label_))


In [ ]:

def extract_contact_number_from_resume(text):
    contact_number = None

    pattern = r"\b(?:\+\d{1,3}\s?)?\(?\d{3}\)?[-.\s]?\d{3,4}[-.\s]?\d{4}\b"
    match = re.search(pattern, text)
    if match:
        contact_number = match.group()

    return contact_number


In [ ]:
extract_contact_number_from_resume(text)

'9817163439'

In [ ]:
def extract_email_from_resume(text):
    email = None

    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    match = re.search(pattern, text)
    if match:
        email = match.group()

    return email

In [ ]:
extract_email_from_resume(text)

'cherubbhujel@gmail.com'

In [ ]:
import spacy
from spacy.matcher import Matcher

In [ ]:
def extract_name(resume_text):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)

    # Define name patterns
    patterns = [
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name, Middle name, and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]  # First name, Middle name, Middle name, and Last name
        # Add more patterns as needed
    ]

    for pattern in patterns:
        matcher.add('NAME', patterns=[pattern])

    doc = nlp(resume_text)
    matches = matcher(doc)

    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text

    return None

In [ ]:
extract_name(text)

'Supriya Thapa'

In [ ]:

def extract_skills_from_resume(text, skills_list):
    skills = []

    for skill in skills_list:
        pattern = r"\b{}\b".format(re.escape(skill))
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            skills.append(skill)

    return skills

In [ ]:

skills_list = ['Python', 'Data Analysis', 'Machine Learning', 'Communication', 'Project Management', 'Deep Learning', 'SQL', 'Tableau']

extracted_skills = extract_skills_from_resume(text, skills_list)

if extracted_skills:
    print("Skills:", extracted_skills)
else:
    print("No skills found")

Skills: ['Python', 'Data Analysis', 'Machine Learning', 'Communication', 'Deep Learning', 'SQL']


In [ ]:
d = pd.read_csv('/home/supriya/Desktop/R-JD-matching/dataset_entities.csv')

In [ ]:
print(d['Text'].iloc[0])

Inova IT Helpdesk Technician Inova IT Helpdesk Technician Inova IT Helpdesk Technician Stafford VA Highly accomplished and experienced Communications Specialist with over 5 years of experience installation configuration and management of data network systems in both a stand-alone and client-server environment. Consistently complete projects with little to no supervision with superior results. Daily planned and executed the integration of multiple information systems in a network environment evaluated and resolved customer information system problems and utilized the required hardware upgrades and repaired to maintain mission capability.  A driven and versatile leader with a record of success in projects requiring an in-depth knowledge and solid experience working with complex communications systems. A quick study in emerging technology able to quickly assess and utilize the appropriate response to meet the specific needs of the business requirement at hand. A welcome addition to any te

In [ ]:
d.head(2)

,Text,Skills,Education,Experience,Additional_Information,Software_Developer,Front_End_Developer,Network_Administrator,Web_Developer,Project_manager,Database_Administrator,Security_Analyst,Systems_Administrator,Python_Developer,Java_Developer,Labels
0,Inova IT Helpdesk Technician Inova IT Helpdesk...,Training Networking (6 years) VOIP (3 years) ...,NaN,Inova IT Helpdesk Technician Inova Healthcare ...,CORE COMPETENCIES Tactical Data Network Perso...,0,0,1,0,0,0,1,0,0,0,"[2, 6]"
1,ServiceNow Administrator/ BA ServiceNow Admini...,NC SQL (6 years) Pharmaceutical Product Devel...,Masters in Computer Science Towson University ...,ServiceNow Administrator/ BA Sabre - Southlake...,Technical Skill Set ITSM Tools ServiceNow Ber...,1,0,0,0,0,0,0,0,1,0,"[8, 0]"


In [ ]:
d.columns

Index(['Text', 'Skills', 'Education', 'Experience', 'Additional_Information',
       'Software_Developer', 'Front_End_Developer', 'Network_Administrator',
       'Web_Developer', 'Project_manager', 'Database_Administrator',
       'Security_Analyst', 'Systems_Administrator', 'Python_Developer',
       'Java_Developer', 'Labels'],
      dtype='object')

In [ ]:
function_1('/home/supriya/Desktop/R-JD-matching/Supriya_Resume.pdf')

OSError: [E050] Can't find model 'generated_model/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.

In [25]:
import spacy 
import en_core_web_sm
nlp = en_core_web_sm.load()

text = extract_text_from_pdf('/home/supriya/Desktop/R-JD-matching/Supriya_Resume.pdf')

x = nlp(text)

for i in x.ents:
    print("Text: ", i.text, '\tLabel:',i.label_)

OSError: [E053] Could not read config file from /home/supriya/anaconda3/lib/python3.11/site-packages/en_core_web_sm/en_core_web_sm-2.3.1/config.cfg